In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [20]:
# Loading test set data 
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [21]:
# Loading items data
item_cat = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [22]:
# Loading shops data
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [23]:
# Loading items list data
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [24]:
# Loading training data
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
train = pd.DataFrame(train)
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
train.shape

(2935849, 6)

In [25]:
# creating a new column product_count that contains the number of products sold per shop per month
new_train = train.groupby(['date_block_num','shop_id','item_id']).agg(product_count= pd.NamedAgg(column='item_cnt_day',aggfunc=sum)).reset_index()

In [26]:
new_train.head()

,date_block_num,shop_id,item_id,product_count
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
model = LinearRegression()

In [29]:
y_train = pd.DataFrame(new_train.iloc[:,-1])

In [15]:
x_train = new_train.iloc[:,:-1]
x_train

,date_block_num,shop_id,item_id
0,0,0,32
1,0,0,33
2,0,0,35
3,0,0,43
4,0,0,51
...,...,...,...
1609119,33,59,22087
1609120,33,59,22088
1609121,33,59,22091
1609122,33,59,22100


In [30]:
#fitting the linear model
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
# Making the predictions
predictions = model.predict(test)
final_df = pd.DataFrame(test.loc[:,"ID"])
final_df['item_cnt_month'] = predictions
final_df.head()

,ID,item_cnt_month
0,0,2.313684
1,1,2.311584
2,2,2.315125
3,3,2.317356
4,4,2.319022


In [18]:
#saving predictions to CSV file
final_df.to_csv("megs_sub1.csv",index=False, header=True)